In [ ]:
#Use hsvw3 env
#Add kernel
#run on Volta notes
#Load 3 modules

import os, time, datetime
import torch
import torch.nn as nn
from sklearn.metrics import roc_curve, auc
from torch.utils.data import DataLoader
from torchvision import models
import sys
import torch.nn.functional as F # noqa

sys.path.append(os.path.dirname(os.getcwd()))
from Py_common.tv_utils import fix_all_seeds_torch
from Py_common.tv_Dataset_strips import ValDataset_strips
from TV.CBAM import  ResNet50_CBAM, predict, find_item,tensor_to_rgb_image, CAM_Manual 
from TV.CBAM import plot_unique_gradient_histgram, plot_gradient_heatmap, select_info, order_informations, plot_individual_maps, get_topk_rankedby_overall_and_specific
from TV.CBAM import LabelProcessor, get_prediction_df, compute_performance, get_sorted_feature_idx_abs, show_cam_on_image
from TV.CBAM import AllImageInfo, get_sorted_score_idxes, normalize_to_minus_one_to_one, symlog_transform, mean_std_norm
import torchvision
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import re
import matplotlib as mpl

In [ ]:
def plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_loc):
    
    for selected_sp in selected_ids:
        #get image
        img, label, sp_id  = find_item(test_ds, selected_sp)
        img = img.to(device)    

        #Get RGB img
        rgb_img = tensor_to_rgb_image(img) #Get RGB img for plot

        #Get patient indx
        pt_idx = all_ids_test.index(selected_sp)

        #Get all info
        cur_grad, cur_score_map, cur_cam_norm_ori, cur_cam_resized, cur_featuremaps, cur_pred_class = image_obj[pt_idx] #NOTE: cur_cam_norm =  normed version of cur_score_map.sum(axis = 0)

        #Get CAM before normlization
        cur_cam = cur_score_map.sum(axis = 0)

        #Get current image specifc scores for each feature
        cur_score_sum = np.array(scores_df[selected_sp]) #this is equal to cur_score_map.sum(axis = (1,2)) 

        #Print overall
#         print('TOP feature overall:', top_oa_idxes)
#         print('TOP score overall:',top_oa_scores)

        #Get top K specific feature
        cur_idxes_sorted = get_sorted_score_idxes(cur_score_sum, use_abs = True)
        top_k = 7
        top_sp_idxes =  cur_idxes_sorted[0:top_k]
        top_sp_scores = cur_score_sum[top_sp_idxes].round(2)
#         print('TOP feature specific:', top_sp_idxes)
#         print('TOP score specific:',top_sp_scores)

        #Get Top specific, but not in overall
        top_sp_idxes2 = [f for f in cur_idxes_sorted if f not in top_oa_idxes][0:top_k]
        top_sp_scores2 = cur_score_sum[top_sp_idxes2].round(2)
#         print('TOP feature specific:2', top_sp_idxes2)
#         print('TOP score specific2:',top_sp_scores2)


        rgb_image = rgb_img
        fig_size = (30,16)
        cmap = 'RdYlBu_r'
        

        #All
        cur_cam2 = cur_cam/cur_cam.max()

        const = cur_score_map.max()
        cur_score_map2 = [x/const for x in cur_score_map]

        #All
        combs = np.append([cur_cam2],cur_score_map2, axis = 0)

        #Norm for acroos all maps
        combs = combs - combs.min() 
        combs = combs/combs.max()

        #select
        scores =  list(cur_score_sum[top_oa_idxes + top_sp_idxes2])
        selected_scores_overall = list(scores_overall[top_oa_idxes + top_sp_idxes2])

        #Modify index
        top_oa_idxes_modified = [x +1 for x in top_oa_idxes]
        top_sp_idxes_modified = [x +1 for x in top_sp_idxes2]
        feature_idxes =  [0] + top_oa_idxes_modified + top_sp_idxes_modified
        plot_array = combs[feature_idxes]


        vis_list = []
        f_idx_list = []
        score_list = []
        score_overall_list = []
        for i in range(len(feature_idxes)):


            #resize
            if i>0:
                cur_score = scores[i-1]
                cur_score_overall = selected_scores_overall[i-1]
            else:
                cur_score = None
                cur_score_overall = None
            heatmap = plot_array[i,]
            heatmap = cv2.resize(heatmap, (rgb_image.shape[1], rgb_image.shape[0]))

            #Overlay heatmap to img
            vis = show_cam_on_image(rgb_image, heatmap, cmap = cmap, use_rgb=False, use_custom_cmap = True, image_weight = 0.5, cam_method = vis_method)

            vis_list.append(vis)
            f_idx_list.append(feature_idxes[i])
            score_list.append(cur_score)
            score_overall_list.append(cur_score_overall)

        # Plot the heatmaps
        vis_list = [rgb_image] + vis_list
        f_idx_list = ['-1'] + f_idx_list
        score_list = ['-1'] + score_list
        score_overall_list = ['-1'] + score_overall_list

        fig, axes = plt.subplots(1, len(vis_list), figsize=fig_size)

        for i, ax in enumerate(axes):
            im = ax.imshow(vis_list[i], cmap=cmap)
            ax.set_xticks([])  # Remove x ticks
            ax.set_yticks([])  # Remove y ticks

            if i == 0:
                ax.set_title('Image \n', fontsize=15)
            elif i == 1:
                ax.set_title('CAM \n', fontsize=15)
            else:
                ax.set_title(f"F{int(f_idx_list[i])-1} \nOA:{float(score_overall_list[i]):.2f} \nSP:{float(score_list[i]):.2f} ", fontsize=15)

        
        # Add a vertical dashed line 
        fig.subplots_adjust(wspace=0.1)  

        # Get the x position of the 11th and 12th subplot
        x_left = axes[int(len(vis_list)/2)].get_position().x1  
        x_right = axes[int(len(vis_list)/2 + 1)].get_position().x0  
        x_middle = (x_left + x_right) / 2  # Middle of the white space

        # Correct way to add a vertical dashed line
        line = mpl.lines.Line2D([x_middle, x_middle], [0.18, 0.9], transform=fig.transFigure,
                                color='black', linestyle='dashed', linewidth=2)
        fig.lines.append(line)  # Add the line to the figure

            
            
        #Normalize colormap since the first image use differnt scale 
        norm = mpl.colors.Normalize(vmin=0, vmax=255)  # 
        sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])  # Empty array, as we only need the colormap

        # Add a shared color bar at the bottom
        cbar = plt.colorbar(sm, ax=axes, orientation='horizontal', fraction=0.046, pad=0.04, extend='neither')
        tick_positions = np.linspace(0, 255, 6)
        tick_labels = np.linspace(-1, 1, 6)
        cbar.set_ticks(tick_positions)
        cbar.set_ticklabels([f"{t:.1f}" for t in tick_labels])  # Format labels from -1 to 1

        save_path4 = os.path.join(save_loc, 'Top9OAandTOP9SP')
        if not os.path.exists(save_path4):
            os.makedirs(save_path4)        
        
        if cur_pred_class != label:
            save_path_mis = os.path.join(save_path4, 'Misclassified')
            
            if not os.path.exists(save_path_mis):
                os.makedirs(save_path_mis)
            final_path = save_path_mis
        else:
            final_path = save_path4 
            
        plt_title =  selected_sp + '\n Predicted Class ' + str(cur_pred_class) +  ' True Class: ' + str(label)
        map_title = 'Score Map\n' + plt_title
        plt.suptitle(map_title, fontsize=17)     
        
        plt.savefig(os.path.join(final_path, selected_sp + "_Score_Map.png"), dpi=300, bbox_inches='tight', pad_inches=0, facecolor='white')
        plt.show()
        plt.close()

## target-section

In [ ]:
gpu_id = 0
input_data_name = "SEG_sS1_strips_v4" #SEG_sS1_strips_v6, SEG_sS1_strips_v4, DET_dS_strips
label_file = '../../Image/sS_labels.csv'
diagnostic_type = 'Final'
num_classes = 2
mask_dir = 'None'
data_aug_ctrl = 1
batch_size = 24
test_img_dir = '../../Image/test_' + input_data_name
model_name = 'CLS_HSV2_Final_2classes_SEG_sS1_strips_v4_pretrained_seed0'
model_path = os.path.join('../../Model/', model_name +'.pth')
out_location = "../../Output/output03092025/"
# Extract the integer after "HSV" from model name
match = re.search(r'HSV(\d+)', model_name)
HSV = int(match.group(1)) if match else None

#Selelcted for heatmap plot
vis_method = 'CAM'

# need to set visibility before defining device
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
### Check whether gpu is available
if torch.cuda.is_available():
    gpu = True
    device = torch.device('cuda')  # this will use the visible gpu
else:
    gpu = False
    device = torch.device('cpu')

# set seeds
fix_all_seeds_torch(gpu_id)

### Set Directory
outdir = os.path.join(out_location, model_name, input_data_name, 'HSV' + str(HSV))
save_path = os.path.join(outdir, "heatmap")
if not os.path.exists(save_path):
    os.makedirs(save_path)



########################################
########     Load test data     ########
########################################
test_ds = ValDataset_strips(img_dir= test_img_dir,
                         label_file=label_file,
                         HSV=HSV,
                         diagnostic_type=diagnostic_type,
                         num_classes=num_classes,
                         mask_dir=None if mask_dir == 'None' else mask_dir,
                         nchan = None # assuming we will supply strip images and mask_dir is None
                )
print(f"Number of samples: {len(test_ds)}")
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
n_batches_test = len(test_loader)

## Construct Model

In [ ]:
########################################
########    Construct Model     ########
########################################
nchan=test_ds.nchan
print(f"nchan: {nchan}")

# Load model
if 'CBAM' in model_name:
    model = ResNet50_CBAM(nchan = nchan, num_classes = num_classes, pretrained_model = "Resnet50_withPretrainedWeight", use_cbam_class = True, reduction_ratio = 1, kernel_cbam = 3, freeze = False)
    cbam_flag = True
else:
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    model.conv1 = nn.Conv2d(nchan, 64, kernel_size=7, stride=2, padding=3, bias=False) # Train1
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    cbam_flag = False
    
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()


# Load the state dictionary into the model
model.load_state_dict(torch.load(model_path))
model.to(device)
print('Model Loaded')

## Predict

In [ ]:
loss_test, accuracy_test, correct_test, auc_test , all_ids_test, all_scores_test, all_labels_test = predict(test_loader,model, device, criterion)

#Load label file
label_df = pd.read_csv(label_file)
label_processor = LabelProcessor(label_df, diagnostic_type)
label_df = label_processor.process_labels()

#Pred
pred_df = get_prediction_df(all_ids_test, all_scores_test, all_labels_test, label_df, pred_thres = 0.5)

## Get All CAM, HEATMAP, GRADIENT, ACTIVATION, FEATURE MAP for all test IDs

In [ ]:
############################################################################################################
# Get All CAM, HEATMAP, GRADIENT, ACTIVATION, FEATURE MAP for all test IDs
############################################################################################################
cam_list = []
heatmap_list = []
gradient_list = []
scores_list = []
feature_map_list = []
pred_class_list =[]

for strip_id in all_ids_test:
    img, label, sp_id  = find_item(test_ds, strip_id)
    img = img.to(device)
    img_tensor = img.unsqueeze(0).to(device)  


    cur_pred_df = pred_df.loc[pred_df['strip_id'] == strip_id]
    cur_pred_class = cur_pred_df['PRED_CLASS'].item()

    #Initializae
    cam_ranka = CAM_Manual(model=model, cam_method = vis_method, opposite_class = False, CBAM_FLAG = False)
    
    #Get cam: avg/sum overall all activation map; heatmap: resized cam, gradients gradients (alphas), and raw_scores (grad*feature map), feature_maps are the feature maps from CNN
    cam, heatmap, gradients, raw_scores, feature_maps = cam_ranka(img_tensor)

    cam_list.append(cam.unsqueeze(0).cpu().numpy())
    heatmap_list.append(np.expand_dims(heatmap, axis=0))
    gradient_list.append(gradients.cpu().numpy())
    scores_list.append(raw_scores.cpu().numpy())
    feature_map_list.append(feature_maps.cpu().numpy()) 
    pred_class_list.append(cur_pred_class)

cam_all = np.vstack(cam_list)
heatmap_all = np.vstack(heatmap_list)
gradient_all = np.vstack(gradient_list)
score_all = np.vstack(scores_list)
feature_map_all = np.vstack(feature_map_list)
pred_class_all = np.vstack(pred_class_list)

print("Gradient:",gradient_all.shape)
print("Scores:",score_all.shape)
print("CAM:",cam_all.shape)
print("Heatmap (resized CAM):",heatmap_all.shape)
print("Feature maps:",feature_map_all.shape)
print("Predicted Class:",pred_class_all.shape)
#Create image object for all test image
image_obj = AllImageInfo(gradient_all, score_all, cam_all, heatmap_all, feature_map_all, pred_class_all)

# sum over axis 2 and 3 of score_all to get the final score
out = score_all.sum(axis=(2,3))
# transpose to make columns correspond to strip ids
out = np.transpose(out)
# save out to a csv file and use all_ids_test as column names
np.savetxt(os.path.join(save_path, 'score_by_feature_testimage.csv'), out, delimiter=',', header=','.join(all_ids_test), comments="")

## Vsiluzation of gradients

In [ ]:
#A. Top 10 Gradients for all strip id
file_path = os.path.join(save_path, "gradient_all_heatmap.png")
plot_gradient_heatmap(gradient_all, 10, pred_class_all, file_path, HSV, vis_method, sort_by_grad = True, sort_by_class = True)

#B. Histograms of Unique Gradient for strip id
grad_unique, idx_unique = np.unique(gradient_all, axis=0, return_index=True)
grad_class = pred_class_all[idx_unique].flatten().tolist()

file_path = os.path.join(save_path, "gradient_unique_hist.png")
plot_unique_gradient_histgram(grad_unique, grad_class, HSV, vis_method, file_path)

#C.TOP 10 (By Abs Gradeints) features for CLASS1 and CLASS0
top_f = 10
top_f_dict = {}
for i in range(2):
    top_f_idxes = np.argsort(abs(grad_unique), axis=1)[i][::-1].tolist()[0:top_f]
    top_f_dict[grad_class[i]] = top_f_idxes
print(top_f_dict)

file_path = os.path.join(save_path, "TOP_Gradient_FeaturesIndex_byClass.txt")
with open(file_path, 'w') as f:
    f.write(str(top_f_dict))

## Get average abs score = sum(grad*feature) over all test samples

In [ ]:
scores_df = pd.read_csv(os.path.join(save_path, 'score_by_feature_testimage.csv'))
scores_overall = np.array(scores_df.abs().mean(axis = 1)) #average over all samples

#Compare
path1 = '/fh/fast/fong_y/HSVW/Feature/CLS_HSV' + str(HSV) + '_Final_2classes_SEG_sS1_strips_v4_pretrained_seed0.pth/score_by_feature_testimage.csv'
check1 = pd.read_csv(path1)

print(check1.equals(scores_df))

## Get top k overall score and index 

In [ ]:
feature_idxes_sorted = get_sorted_score_idxes(scores_overall, use_abs = True)
top_k = 7
top_oa_idxes = feature_idxes_sorted[0:top_k]
top_oa_scores = scores_overall[top_oa_idxes].round(3)
print('TOP feature overall:', top_oa_idxes)
print('TOP score overall:',top_oa_scores)

### Plot CAM and indiviudal score maps
#### Normalization Note:
 - because CAM is not in the same scale as the rest of score map
 -  1st:  norm CAM , CAM/CAM.max() 
 -  2nd: norm other heatmaps, heatmap_i/all_heatmaps.max()
 -  3rd: norm to 0 to 1 altogether, now all color scale are fine

## 1. HSV1+, HSV2- (Correct)

In [ ]:
#Get IDs
grp = 'HSV1POS_HSV2NEG'
cond = (pred_df['GROUP'] == grp) & (pred_df['Label'] == pred_df['PRED_CLASS'])
selected_ids = list(pred_df.loc[cond, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))

#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 2. HSV1+, HSV2+ (Correct)

In [ ]:
#Get IDs
grp = 'HSV1POS_HSV2POS'
selected_ids = list(pred_df.loc[pred_df['GROUP'] == grp, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))


#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 3. HSV1-, HSV2+ (Correct)

In [ ]:
#Get IDs
grp = 'HSV1NEG_HSV2POS'
selected_ids = list(pred_df.loc[pred_df['GROUP'] == grp, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))


#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 4. HSV1-, HSV2- (Correct)

In [ ]:
#Get IDs
grp = 'HSV1NEG_HSV2NEG'
selected_ids = list(pred_df.loc[pred_df['GROUP'] == grp, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))


#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 1. HSV1+, HSV2- (Misclassified)

In [ ]:
grp = 'HSV1POS_HSV2NEG'
cond = (pred_df['GROUP'] == grp) & (pred_df['Label'] != pred_df['PRED_CLASS'])
selected_ids = list(pred_df.loc[cond, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))

#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 2. HSV1+, HSV2+  (Misclassified)

In [ ]:
grp = 'HSV1POS_HSV2POS'
cond = (pred_df['GROUP'] == grp) & (pred_df['Label'] != pred_df['PRED_CLASS'])
selected_ids = list(pred_df.loc[cond, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))

#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 3. HSV1-, HSV2+  (Misclassified)

In [ ]:
grp = 'HSV1NEG_HSV2POS'
cond = (pred_df['GROUP'] == grp) & (pred_df['Label'] != pred_df['PRED_CLASS'])
selected_ids = list(pred_df.loc[cond, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))

#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

## 4. HSV1-, HSV2-  (Misclassified)

In [ ]:
grp = 'HSV1NEG_HSV2NEG'
cond = (pred_df['GROUP'] == grp) & (pred_df['Label'] != pred_df['PRED_CLASS'])
selected_ids = list(pred_df.loc[cond, 'strip_id'])
print('Plot for ' + grp + ": " + str(len(selected_ids)))

#Create output folder
save_path2 = os.path.join(save_path, grp)
if not os.path.exists(save_path2):
    os.makedirs(save_path2)

plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path2)

### Plot for other cases

In [ ]:
selected_ids = ['2016.09.22_CZ_03_217','2016.09.01_CZ_01_251','2016.09.22_CZ_02_239','2016.09.22_CZ_03_215', '2016.09.22_CZ_03_213', '2016.09.22_CZ_03_211', '2016.09.01_CZ_03_249', '2016.10.31_CZ_01_231' ]
save_path6 = os.path.join(outdir, "heatmap", "other_exp")
if not os.path.exists(save_path6):
    os.makedirs(save_path6)
plot_function(selected_ids,test_ds,image_obj,scores_df,top_oa_idxes,top_oa_scores,save_path6)